In [39]:
import pandas as pd
import chardet
import nltk
from nltk.corpus import stopwords
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
from twitter_program import connect_Twitter,tweet_clean_tp,tweet_parse_tp,retrieve_hashtags,retrieve_mentions,retrieve_urls,pos_tags,extract_emojis,spell_check,lemmatise
from nltk.stem import PorterStemmer
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd
import json
import gc
import statistics
from datetime import datetime, timedelta

In [2]:
def preprocessing(String):
    
    '''
    This function is used for preprocessing
    - Tokenization
    - Stemming
    - Stop Words
        
    '''
    tokens = nltk.word_tokenize(String)
    token = [word for word in tokens if word.isalpha()]
    influential_words = " ".join(token)
    influential_words = influential_words.lower()
    influential_words = influential_words.split()
    stemwords_string = " ".join(influential_words)
    return stemwords_string

In [3]:
model_10 = r"C:\Users\Manjusha Pattadkal\Documents\ML Project\Sentiment Analysis\model_10"
loaded_tokenizer = DistilBertTokenizer.from_pretrained(model_10)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(model_10)

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at C:\Users\Manjusha Pattadkal\Documents\ML Project\Sentiment Analysis\model_10.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [4]:
import os
os.getcwd()

'C:\\Users\\Manjusha Pattadkal'

In [141]:
data = pd.read_csv(r'C:\Users\Manjusha Pattadkal\Downloads\depTweetsOrderedByDate.csv')
data_new = data[['masked_user_id', 'tweets', 'new_date']].copy()
data_sorted=data_new.sort_values(by=['masked_user_id', 'new_date'])
data_sorted=data_sorted.reset_index()

In [153]:
def start_end_date(data, date_column_name = "timestamp"):
    """
    The function returns the start date and the end date of the tweets by an individual 
    
    Parameters
    ----------
    data : TYPE
        DESCRIPTION.
    date_column_name : TYPE, optional
        DESCRIPTION. The default is "timestamp".

    Returns
    -------
    start_date : TYPE
        DESCRIPTION.
    end_date : TYPE
        DESCRIPTION.

    """
   
   # data[date_column_name] = data[date_column_name].astype('datetime64[ns]')
    #data[date_column_name]=pd.to_datetime(data[date_column_name], errors='coerce',dayfirst=True,format= "%d-%m-%y %H:%M")
    end_date = data[date_column_name].max()
    start_date = data[date_column_name].min()
    return start_date, end_date

In [143]:
def to_df(dates_1,score):
    df = pd.DataFrame({'timestamp': dates_1, "Sentiment score": score})
    return df

In [145]:
user = "depressed_user_1"
user_tweet = []
date_tweet = []
prediction_value = []
final_score = []
for i in range(0,len(data_sorted)):
    

    if user == data_sorted.iloc[i]["masked_user_id"]:
        processed=str(preprocessing(str(data_sorted.iloc[i]["tweets"])))
        
        predict_input = loaded_tokenizer.encode(processed,
                                 truncation=True,
                                 pad_to_max_length=True,
                                 return_tensors="tf")
        output = loaded_model(predict_input)[0]
        
        #output=tf.nn.softmax(output)
        prediction_value.append(tf.argmax(output, axis=1, output_type=tf.int32))
        #print(tf.argmax(output))
        date_tweet.append(data_sorted.iloc[i]["new_date"])
        
        if i!=len(data_sorted)-1:
            continue
    
    date_new_tweet = pd.DataFrame(date_tweet)
   # print(date_new_tweet)
    date_new_tweet[0]=pd.to_datetime(date_new_tweet[0], errors='coerce',dayfirst=True)
   # print(date_new_tweet)
    tweet_df = to_df(date_new_tweet[0],prediction_value)
    start,end =start_end_date(tweet_df)
    #print(start,end)
    
    #start =datetime.strptime(date_tweet[0], "%d-%m-%yyyy %H:%M")
    
    inter = start + timedelta(days=7)
    #print(start,inter)
    
    vec = []
    for l in range(0,8):
            print(start,inter,end)
            postive_count = 0
            negative_count = 0
            count = 0
            for k in range(0,len(tweet_df)):
                try:
                    if tweet_df.iloc[k][0] >= start and tweet_df.iloc[k][0]<=inter:
                        if prediction_value[k]==4:
                            postive_count= postive_count+1
                        if prediction_value[k]==0:
                            negative_count= negative_count+1
                        count = count + 1
                except:
                    pass
            if count!=0:
                vec.append(postive_count/count)
                vec.append(negative_count/count)
            else:
                vec.append(0)
            
            start = inter
            inter = start + timedelta(days=7)
    vec.extend("1")
    final_score.append(vec)
    
    user =  data_sorted.iloc[i]["masked_user_id"]
    prediction_value=[]
    date_tweet=[]
    processed=str(preprocessing(str(data_sorted.iloc[i]["tweets"])))
        
    predict_input = loaded_tokenizer.encode(processed,
                                 truncation=True,
                                 pad_to_max_length=True,
                                 return_tensors="tf")
    output = loaded_model(predict_input)[0]
    output=tf.nn.softmax(output)
    prediction_value.append(tf.reduce_max(output))
    date_tweet.append(data_sorted.iloc[i]["new_date"])

2020-07-09 11:18:00 2020-07-16 11:18:00 2020-09-09 20:03:00
2020-07-16 11:18:00 2020-07-23 11:18:00 2020-09-09 20:03:00
2020-07-23 11:18:00 2020-07-30 11:18:00 2020-09-09 20:03:00
2020-07-30 11:18:00 2020-08-06 11:18:00 2020-09-09 20:03:00
2020-08-06 11:18:00 2020-08-13 11:18:00 2020-09-09 20:03:00
2020-08-13 11:18:00 2020-08-20 11:18:00 2020-09-09 20:03:00
2020-08-20 11:18:00 2020-08-27 11:18:00 2020-09-09 20:03:00
2020-08-27 11:18:00 2020-09-03 11:18:00 2020-09-09 20:03:00
2020-07-20 21:32:00 2020-07-27 21:32:00 2020-09-16 01:54:00
2020-07-27 21:32:00 2020-08-03 21:32:00 2020-09-16 01:54:00
2020-08-03 21:32:00 2020-08-10 21:32:00 2020-09-16 01:54:00
2020-08-10 21:32:00 2020-08-17 21:32:00 2020-09-16 01:54:00
2020-08-17 21:32:00 2020-08-24 21:32:00 2020-09-16 01:54:00
2020-08-24 21:32:00 2020-08-31 21:32:00 2020-09-16 01:54:00
2020-08-31 21:32:00 2020-09-07 21:32:00 2020-09-16 01:54:00
2020-09-07 21:32:00 2020-09-14 21:32:00 2020-09-16 01:54:00
2020-07-16 02:38:00 2020-07-23 02:38:00 

2020-07-08 00:00:00 2020-07-15 00:00:00 2020-09-08 13:26:00
2020-07-15 00:00:00 2020-07-22 00:00:00 2020-09-08 13:26:00
2020-07-22 00:00:00 2020-07-29 00:00:00 2020-09-08 13:26:00
2020-07-29 00:00:00 2020-08-05 00:00:00 2020-09-08 13:26:00
2020-08-05 00:00:00 2020-08-12 00:00:00 2020-09-08 13:26:00
2020-08-12 00:00:00 2020-08-19 00:00:00 2020-09-08 13:26:00
2020-08-19 00:00:00 2020-08-26 00:00:00 2020-09-08 13:26:00
2020-08-26 00:00:00 2020-09-02 00:00:00 2020-09-08 13:26:00
2020-07-28 01:30:00 2020-08-04 01:30:00 2020-09-27 04:56:00
2020-08-04 01:30:00 2020-08-11 01:30:00 2020-09-27 04:56:00
2020-08-11 01:30:00 2020-08-18 01:30:00 2020-09-27 04:56:00
2020-08-18 01:30:00 2020-08-25 01:30:00 2020-09-27 04:56:00
2020-08-25 01:30:00 2020-09-01 01:30:00 2020-09-27 04:56:00
2020-09-01 01:30:00 2020-09-08 01:30:00 2020-09-27 04:56:00
2020-09-08 01:30:00 2020-09-15 01:30:00 2020-09-27 04:56:00
2020-09-15 01:30:00 2020-09-22 01:30:00 2020-09-27 04:56:00
2020-07-28 17:28:00 2020-08-04 17:28:00 

2020-08-06 21:40:00 2020-08-13 21:40:00 2020-09-09 15:52:00
2020-08-13 21:40:00 2020-08-20 21:40:00 2020-09-09 15:52:00
2020-08-20 21:40:00 2020-08-27 21:40:00 2020-09-09 15:52:00
2020-08-27 21:40:00 2020-09-03 21:40:00 2020-09-09 15:52:00
2020-07-16 17:10:00 2020-07-23 17:10:00 2020-09-14 16:42:00
2020-07-23 17:10:00 2020-07-30 17:10:00 2020-09-14 16:42:00
2020-07-30 17:10:00 2020-08-06 17:10:00 2020-09-14 16:42:00
2020-08-06 17:10:00 2020-08-13 17:10:00 2020-09-14 16:42:00
2020-08-13 17:10:00 2020-08-20 17:10:00 2020-09-14 16:42:00
2020-08-20 17:10:00 2020-08-27 17:10:00 2020-09-14 16:42:00
2020-08-27 17:10:00 2020-09-03 17:10:00 2020-09-14 16:42:00
2020-09-03 17:10:00 2020-09-10 17:10:00 2020-09-14 16:42:00
2020-07-19 01:05:00 2020-07-26 01:05:00 2020-09-15 19:52:00
2020-07-26 01:05:00 2020-08-02 01:05:00 2020-09-15 19:52:00
2020-08-02 01:05:00 2020-08-09 01:05:00 2020-09-15 19:52:00
2020-08-09 01:05:00 2020-08-16 01:05:00 2020-09-15 19:52:00
2020-08-16 01:05:00 2020-08-23 01:05:00 

2020-08-21 00:31:00 2020-08-28 00:31:00 2020-09-10 00:30:00
2020-08-28 00:31:00 2020-09-04 00:31:00 2020-09-10 00:30:00
2020-07-28 19:44:00 2020-08-04 19:44:00 2020-09-15 16:59:00
2020-08-04 19:44:00 2020-08-11 19:44:00 2020-09-15 16:59:00
2020-08-11 19:44:00 2020-08-18 19:44:00 2020-09-15 16:59:00
2020-08-18 19:44:00 2020-08-25 19:44:00 2020-09-15 16:59:00
2020-08-25 19:44:00 2020-09-01 19:44:00 2020-09-15 16:59:00
2020-09-01 19:44:00 2020-09-08 19:44:00 2020-09-15 16:59:00
2020-09-08 19:44:00 2020-09-15 19:44:00 2020-09-15 16:59:00
2020-09-15 19:44:00 2020-09-22 19:44:00 2020-09-15 16:59:00
2020-07-28 01:39:00 2020-08-04 01:39:00 2020-09-21 15:39:00
2020-08-04 01:39:00 2020-08-11 01:39:00 2020-09-21 15:39:00
2020-08-11 01:39:00 2020-08-18 01:39:00 2020-09-21 15:39:00
2020-08-18 01:39:00 2020-08-25 01:39:00 2020-09-21 15:39:00
2020-08-25 01:39:00 2020-09-01 01:39:00 2020-09-21 15:39:00
2020-09-01 01:39:00 2020-09-08 01:39:00 2020-09-21 15:39:00
2020-09-08 01:39:00 2020-09-15 01:39:00 

2020-07-09 00:33:00 2020-07-16 00:33:00 2020-09-09 23:05:00
2020-07-16 00:33:00 2020-07-23 00:33:00 2020-09-09 23:05:00
2020-07-23 00:33:00 2020-07-30 00:33:00 2020-09-09 23:05:00
2020-07-30 00:33:00 2020-08-06 00:33:00 2020-09-09 23:05:00
2020-08-06 00:33:00 2020-08-13 00:33:00 2020-09-09 23:05:00
2020-08-13 00:33:00 2020-08-20 00:33:00 2020-09-09 23:05:00
2020-08-20 00:33:00 2020-08-27 00:33:00 2020-09-09 23:05:00
2020-08-27 00:33:00 2020-09-03 00:33:00 2020-09-09 23:05:00
2020-07-28 17:19:00 2020-08-04 17:19:00 2020-09-27 05:48:00
2020-08-04 17:19:00 2020-08-11 17:19:00 2020-09-27 05:48:00
2020-08-11 17:19:00 2020-08-18 17:19:00 2020-09-27 05:48:00
2020-08-18 17:19:00 2020-08-25 17:19:00 2020-09-27 05:48:00
2020-08-25 17:19:00 2020-09-01 17:19:00 2020-09-27 05:48:00
2020-09-01 17:19:00 2020-09-08 17:19:00 2020-09-27 05:48:00
2020-09-08 17:19:00 2020-09-15 17:19:00 2020-09-27 05:48:00
2020-09-15 17:19:00 2020-09-22 17:19:00 2020-09-27 05:48:00
2020-07-16 02:46:00 2020-07-23 02:46:00 

2020-07-23 11:55:00 2020-07-30 11:55:00 2020-09-09 17:20:00
2020-07-30 11:55:00 2020-08-06 11:55:00 2020-09-09 17:20:00
2020-08-06 11:55:00 2020-08-13 11:55:00 2020-09-09 17:20:00
2020-08-13 11:55:00 2020-08-20 11:55:00 2020-09-09 17:20:00
2020-08-20 11:55:00 2020-08-27 11:55:00 2020-09-09 17:20:00
2020-08-27 11:55:00 2020-09-03 11:55:00 2020-09-09 17:20:00
2020-07-09 00:01:00 2020-07-16 00:01:00 2020-09-09 16:23:00
2020-07-16 00:01:00 2020-07-23 00:01:00 2020-09-09 16:23:00
2020-07-23 00:01:00 2020-07-30 00:01:00 2020-09-09 16:23:00
2020-07-30 00:01:00 2020-08-06 00:01:00 2020-09-09 16:23:00
2020-08-06 00:01:00 2020-08-13 00:01:00 2020-09-09 16:23:00
2020-08-13 00:01:00 2020-08-20 00:01:00 2020-09-09 16:23:00
2020-08-20 00:01:00 2020-08-27 00:01:00 2020-09-09 16:23:00
2020-08-27 00:01:00 2020-09-03 00:01:00 2020-09-09 16:23:00
2020-07-08 11:44:00 2020-07-15 11:44:00 2020-09-06 22:46:00
2020-07-15 11:44:00 2020-07-22 11:44:00 2020-09-06 22:46:00
2020-07-22 11:44:00 2020-07-29 11:44:00 

In [156]:
final_score

[[0.8048780487804879,
  0.1951219512195122,
  0.8872180451127819,
  0.11278195488721804,
  0.7982456140350878,
  0.20175438596491227,
  0.7943262411347518,
  0.20567375886524822,
  0.927536231884058,
  0.07246376811594203,
  0.7478260869565218,
  0.25217391304347825,
  0.7548387096774194,
  0.24516129032258063,
  0.8848484848484849,
  0.11515151515151516,
  '1'],
 [0.8461538461538461,
  0.15384615384615385,
  0.8064516129032258,
  0.16129032258064516,
  0.9166666666666666,
  0.08333333333333333,
  1.0,
  0.0,
  0.5384615384615384,
  0.46153846153846156,
  0.875,
  0.125,
  0.8421052631578947,
  0.15789473684210525,
  0.8181818181818182,
  0.18181818181818182,
  '1'],
 [0.5454545454545454,
  0.45454545454545453,
  0.56,
  0.44,
  0.6293706293706294,
  0.36363636363636365,
  0.64375,
  0.35625,
  0.6767676767676768,
  0.32323232323232326,
  0.6803118908382066,
  0.31968810916179335,
  0.6743243243243243,
  0.3256756756756757,
  0.6895306859205776,
  0.3104693140794224,
  '1'],
 [0.666666

In [154]:
data = pd.read_csv(r'C:\Users\Manjusha Pattadkal\Downloads\nondepTweetsOrderedByDate.csv')
data_new = data[['masked_user_id', 'tweets', 'new_date']].copy()
data_sorted=data_new.sort_values(by=['masked_user_id', 'new_date'])
data_sorted=data_sorted.reset_index()

In [157]:
final_score_df = pd.DataFrame(final_score)
final_score_df.to_csv(r"C:\Users\Manjusha Pattadkal\Documents\ML Project\sentiment_percent_vector.csv")

In [158]:
final_score_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.804878,0.195122,0.887218,0.112782,0.798246,0.201754,0.794326,0.205674,0.927536,0.072464,0.747826,0.252174,0.754839,0.245161,0.884848,0.115152,1
1,0.846154,0.153846,0.806452,0.161290,0.916667,0.083333,1.000000,0.000000,0.538462,0.461538,0.875000,0.125,0.842105,0.157895,0.818182,0.181818,1
2,0.545455,0.454545,0.560000,0.440000,0.629371,0.363636,0.643750,0.356250,0.676768,0.323232,0.680312,0.319688,0.674324,0.325676,0.689531,0.310469,1
3,0.666667,0.000000,0.800000,0.200000,0.500000,0.500000,0.750000,0.250000,0.600000,0.400000,0.916667,0.0833333,0.8,0.2,0.785714,0.214286,1
4,0.571429,0.428571,0.572368,0.421053,0.621429,0.378571,0.565217,0.434783,0.685864,0.314136,0.589147,0.410853,0.658182,0.341818,0.565517,0.434483,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.808824,0.176471,0.750000,0.250000,0.850000,0.150000,0.470588,0.529412,0.685714,0.314286,0.846154,0.153846,0.725,0.275,0.722222,0.277778,0
197,0.785455,0.210909,0.631579,0.368421,0.780000,0.220000,0.691589,0.308411,0.757009,0.242991,0.748344,0.251656,0.760274,0.239726,0.760736,0.239264,0
198,0.728814,0.254237,0.723077,0.276923,0.764706,0.235294,0.731343,0.268657,0.761905,0.238095,0.660377,0.339623,0.687023,0.312977,0.692308,0.307692,0
199,0.602941,0.389706,0.582090,0.417910,0.627451,0.372549,0.625000,0.375000,0.589041,0.410959,0.500000,0.5,0.518519,0.481481,0.64539,0.35461,0


In [155]:
user = "nondepressed_user_1"
user_tweet = []
date_tweet = []
prediction_value = []

for i in range(0,len(data_sorted)):
    

    if user == data_sorted.iloc[i]["masked_user_id"]:
        processed=str(preprocessing(str(data_sorted.iloc[i]["tweets"])))
        
        predict_input = loaded_tokenizer.encode(processed,
                                 truncation=True,
                                 pad_to_max_length=True,
                                 return_tensors="tf")
        output = loaded_model(predict_input)[0]
        
        #output=tf.nn.softmax(output)
        prediction_value.append(tf.argmax(output, axis=1, output_type=tf.int32))
        #print(tf.argmax(output))
        date_tweet.append(data_sorted.iloc[i]["new_date"])
        
        if i!=len(data_sorted)-1:
            continue
    
    date_new_tweet = pd.DataFrame(date_tweet)
   # print(date_new_tweet)
    date_new_tweet[0]=pd.to_datetime(date_new_tweet[0], errors='coerce')
   # print(date_new_tweet)
    tweet_df = to_df(date_new_tweet[0],prediction_value)
    start,end =start_end_date(tweet_df)
    #print(start,end)
    
    
    #start =datetime.strptime(date_tweet[0], "%d-%m-%yyyy %H:%M")
    
    inter = start + timedelta(days=7)
    #print(start,inter)
    #break
    vec = []
    for l in range(0,8):
            print(start,inter,end)
            postive_count = 0
            negative_count = 0
            count = 0
            for k in range(0,len(tweet_df)):
                try:
                    if tweet_df.iloc[k][0] >= start and tweet_df.iloc[k][0]<=inter:
                        if prediction_value[k]==4:
                            postive_count= postive_count+1
                        if prediction_value[k]==0:
                            negative_count= negative_count+1
                        count = count + 1
                except:
                    pass
            if count!=0:
                vec.append(postive_count/count)
                vec.append(negative_count/count)
            else:
                vec.append(0)
            
            start = inter
            inter = start + timedelta(days=7)
    vec.extend("0")
    final_score.append(vec)
    
    user =  data_sorted.iloc[i]["masked_user_id"]
    prediction_value=[]
    date_tweet=[]
    processed=str(preprocessing(str(data_sorted.iloc[i]["tweets"])))
        
    predict_input = loaded_tokenizer.encode(processed,
                                 truncation=True,
                                 pad_to_max_length=True,
                                 return_tensors="tf")
    output = loaded_model(predict_input)[0]
    output=tf.nn.softmax(output)
    prediction_value.append(tf.reduce_max(output))
    date_tweet.append(data_sorted.iloc[i]["new_date"])

C:\Users\Manjusha Pattadkal\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


2020-07-15 01:28:00 2020-07-22 01:28:00 2020-08-28 15:17:00
2020-07-22 01:28:00 2020-07-29 01:28:00 2020-08-28 15:17:00
2020-07-29 01:28:00 2020-08-05 01:28:00 2020-08-28 15:17:00
2020-08-05 01:28:00 2020-08-12 01:28:00 2020-08-28 15:17:00
2020-08-12 01:28:00 2020-08-19 01:28:00 2020-08-28 15:17:00
2020-08-19 01:28:00 2020-08-26 01:28:00 2020-08-28 15:17:00
2020-08-26 01:28:00 2020-09-02 01:28:00 2020-08-28 15:17:00
2020-09-02 01:28:00 2020-09-09 01:28:00 2020-08-28 15:17:00
2020-07-26 00:10:00 2020-08-02 00:10:00 2020-09-26 12:19:00
2020-08-02 00:10:00 2020-08-09 00:10:00 2020-09-26 12:19:00
2020-08-09 00:10:00 2020-08-16 00:10:00 2020-09-26 12:19:00
2020-08-16 00:10:00 2020-08-23 00:10:00 2020-09-26 12:19:00
2020-08-23 00:10:00 2020-08-30 00:10:00 2020-09-26 12:19:00
2020-08-30 00:10:00 2020-09-06 00:10:00 2020-09-26 12:19:00
2020-09-06 00:10:00 2020-09-13 00:10:00 2020-09-26 12:19:00
2020-09-13 00:10:00 2020-09-20 00:10:00 2020-09-26 12:19:00
2020-08-03 07:20:00 2020-08-10 07:20:00 

2020-07-26 12:13:00 2020-08-02 12:13:00 2020-09-26 04:27:00
2020-08-02 12:13:00 2020-08-09 12:13:00 2020-09-26 04:27:00
2020-08-09 12:13:00 2020-08-16 12:13:00 2020-09-26 04:27:00
2020-08-16 12:13:00 2020-08-23 12:13:00 2020-09-26 04:27:00
2020-08-23 12:13:00 2020-08-30 12:13:00 2020-09-26 04:27:00
2020-08-30 12:13:00 2020-09-06 12:13:00 2020-09-26 04:27:00
2020-09-06 12:13:00 2020-09-13 12:13:00 2020-09-26 04:27:00
2020-09-13 12:13:00 2020-09-20 12:13:00 2020-09-26 04:27:00
2020-08-03 00:05:00 2020-08-10 00:05:00 2020-10-01 23:11:00
2020-08-10 00:05:00 2020-08-17 00:05:00 2020-10-01 23:11:00
2020-08-17 00:05:00 2020-08-24 00:05:00 2020-10-01 23:11:00
2020-08-24 00:05:00 2020-08-31 00:05:00 2020-10-01 23:11:00
2020-08-31 00:05:00 2020-09-07 00:05:00 2020-10-01 23:11:00
2020-09-07 00:05:00 2020-09-14 00:05:00 2020-10-01 23:11:00
2020-09-14 00:05:00 2020-09-21 00:05:00 2020-10-01 23:11:00
2020-09-21 00:05:00 2020-09-28 00:05:00 2020-10-01 23:11:00
2020-08-03 01:01:00 2020-08-10 01:01:00 

2020-08-04 00:07:00 2020-08-11 00:07:00 2020-10-05 01:27:00
2020-08-11 00:07:00 2020-08-18 00:07:00 2020-10-05 01:27:00
2020-08-18 00:07:00 2020-08-25 00:07:00 2020-10-05 01:27:00
2020-08-25 00:07:00 2020-09-01 00:07:00 2020-10-05 01:27:00
2020-09-01 00:07:00 2020-09-08 00:07:00 2020-10-05 01:27:00
2020-09-08 00:07:00 2020-09-15 00:07:00 2020-10-05 01:27:00
2020-09-15 00:07:00 2020-09-22 00:07:00 2020-10-05 01:27:00
2020-09-22 00:07:00 2020-09-29 00:07:00 2020-10-05 01:27:00
2020-08-04 02:29:00 2020-08-11 02:29:00 2020-10-03 22:46:00
2020-08-11 02:29:00 2020-08-18 02:29:00 2020-10-03 22:46:00
2020-08-18 02:29:00 2020-08-25 02:29:00 2020-10-03 22:46:00
2020-08-25 02:29:00 2020-09-01 02:29:00 2020-10-03 22:46:00
2020-09-01 02:29:00 2020-09-08 02:29:00 2020-10-03 22:46:00
2020-09-08 02:29:00 2020-09-15 02:29:00 2020-10-03 22:46:00
2020-09-15 02:29:00 2020-09-22 02:29:00 2020-10-03 22:46:00
2020-09-22 02:29:00 2020-09-29 02:29:00 2020-10-03 22:46:00
2020-07-26 09:46:00 2020-08-02 09:46:00 

2020-08-02 01:30:00 2020-08-09 01:30:00 2020-09-26 05:09:00
2020-08-09 01:30:00 2020-08-16 01:30:00 2020-09-26 05:09:00
2020-08-16 01:30:00 2020-08-23 01:30:00 2020-09-26 05:09:00
2020-08-23 01:30:00 2020-08-30 01:30:00 2020-09-26 05:09:00
2020-08-30 01:30:00 2020-09-06 01:30:00 2020-09-26 05:09:00
2020-09-06 01:30:00 2020-09-13 01:30:00 2020-09-26 05:09:00
2020-09-13 01:30:00 2020-09-20 01:30:00 2020-09-26 05:09:00
2020-07-10 01:08:00 2020-07-17 01:08:00 2020-09-02 22:44:00
2020-07-17 01:08:00 2020-07-24 01:08:00 2020-09-02 22:44:00
2020-07-24 01:08:00 2020-07-31 01:08:00 2020-09-02 22:44:00
2020-07-31 01:08:00 2020-08-07 01:08:00 2020-09-02 22:44:00
2020-08-07 01:08:00 2020-08-14 01:08:00 2020-09-02 22:44:00
2020-08-14 01:08:00 2020-08-21 01:08:00 2020-09-02 22:44:00
2020-08-21 01:08:00 2020-08-28 01:08:00 2020-09-02 22:44:00
2020-08-28 01:08:00 2020-09-04 01:08:00 2020-09-02 22:44:00
2020-08-05 04:01:00 2020-08-12 04:01:00 2020-10-05 06:25:00
2020-08-12 04:01:00 2020-08-19 04:01:00 

2020-08-17 01:36:00 2020-08-24 01:36:00 2020-10-02 20:56:00
2020-08-24 01:36:00 2020-08-31 01:36:00 2020-10-02 20:56:00
2020-08-31 01:36:00 2020-09-07 01:36:00 2020-10-02 20:56:00
2020-09-07 01:36:00 2020-09-14 01:36:00 2020-10-02 20:56:00
2020-09-14 01:36:00 2020-09-21 01:36:00 2020-10-02 20:56:00
2020-09-21 01:36:00 2020-09-28 01:36:00 2020-10-02 20:56:00
2020-07-29 21:25:00 2020-08-05 21:25:00 2020-09-23 21:55:00
2020-08-05 21:25:00 2020-08-12 21:25:00 2020-09-23 21:55:00
2020-08-12 21:25:00 2020-08-19 21:25:00 2020-09-23 21:55:00
2020-08-19 21:25:00 2020-08-26 21:25:00 2020-09-23 21:55:00
2020-08-26 21:25:00 2020-09-02 21:25:00 2020-09-23 21:55:00
2020-09-02 21:25:00 2020-09-09 21:25:00 2020-09-23 21:55:00
2020-09-09 21:25:00 2020-09-16 21:25:00 2020-09-23 21:55:00
2020-09-16 21:25:00 2020-09-23 21:25:00 2020-09-23 21:55:00
2020-08-04 00:00:00 2020-08-11 00:00:00 2020-10-05 02:09:00
2020-08-11 00:00:00 2020-08-18 00:00:00 2020-10-05 02:09:00
2020-08-18 00:00:00 2020-08-25 00:00:00 

2020-08-25 00:09:00 2020-09-01 00:09:00 2020-10-05 14:35:00
2020-09-01 00:09:00 2020-09-08 00:09:00 2020-10-05 14:35:00
2020-09-08 00:09:00 2020-09-15 00:09:00 2020-10-05 14:35:00
2020-09-15 00:09:00 2020-09-22 00:09:00 2020-10-05 14:35:00
2020-09-22 00:09:00 2020-09-29 00:09:00 2020-10-05 14:35:00
2020-08-03 21:20:00 2020-08-10 21:20:00 2020-10-03 15:53:00
2020-08-10 21:20:00 2020-08-17 21:20:00 2020-10-03 15:53:00
2020-08-17 21:20:00 2020-08-24 21:20:00 2020-10-03 15:53:00
2020-08-24 21:20:00 2020-08-31 21:20:00 2020-10-03 15:53:00
2020-08-31 21:20:00 2020-09-07 21:20:00 2020-10-03 15:53:00
2020-09-07 21:20:00 2020-09-14 21:20:00 2020-10-03 15:53:00
2020-09-14 21:20:00 2020-09-21 21:20:00 2020-10-03 15:53:00
2020-09-21 21:20:00 2020-09-28 21:20:00 2020-10-03 15:53:00
2020-08-11 22:19:00 2020-08-18 22:19:00 2020-10-04 23:02:00
2020-08-18 22:19:00 2020-08-25 22:19:00 2020-10-04 23:02:00
2020-08-25 22:19:00 2020-09-01 22:19:00 2020-10-04 23:02:00
2020-09-01 22:19:00 2020-09-08 22:19:00 

In [122]:
output

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[ -0.19602527, -15.79173   , -15.629073  , -15.412304  ,
          3.6281302 ]], dtype=float32)>

In [97]:
type(date_tweet[0])

str

In [81]:
df = to_df(date_new_tweet,prediction_value)

In [160]:
i will take this keyboard

[[0.8048780487804879,
  0.1951219512195122,
  0.8872180451127819,
  0.11278195488721804,
  0.7982456140350878,
  0.20175438596491227,
  0.7943262411347518,
  0.20567375886524822,
  0.927536231884058,
  0.07246376811594203,
  0.7478260869565218,
  0.25217391304347825,
  0.7548387096774194,
  0.24516129032258063,
  0.8848484848484849,
  0.11515151515151516,
  '1'],
 [0.8461538461538461,
  0.15384615384615385,
  0.8064516129032258,
  0.16129032258064516,
  0.9166666666666666,
  0.08333333333333333,
  1.0,
  0.0,
  0.5384615384615384,
  0.46153846153846156,
  0.875,
  0.125,
  0.8421052631578947,
  0.15789473684210525,
  0.8181818181818182,
  0.18181818181818182,
  '1'],
 [0.5454545454545454,
  0.45454545454545453,
  0.56,
  0.44,
  0.6293706293706294,
  0.36363636363636365,
  0.64375,
  0.35625,
  0.6767676767676768,
  0.32323232323232326,
  0.6803118908382066,
  0.31968810916179335,
  0.6743243243243243,
  0.3256756756756757,
  0.6895306859205776,
  0.3104693140794224,
  '1'],
 [0.666666

In [56]:
tweet_df = pd.DataFrame({'timestamp': date_tweet, "Sentiment score": prediction_value})

In [57]:
tweet_df

,timestamp,Sentiment score
0,01-08-2020 14:01,"tf.Tensor(0.97862977, shape=(), dtype=float32)"
1,01-08-2020 14:31,"tf.Tensor(0.927323, shape=(), dtype=float32)"
2,01-08-2020 14:32,"tf.Tensor(0.76076806, shape=(), dtype=float32)"
3,01-08-2020 14:35,"tf.Tensor(0.9898533, shape=(), dtype=float32)"
4,01-08-2020 14:44,"tf.Tensor(0.92802703, shape=(), dtype=float32)"
...,...,...
1005,31-08-2020 19:36,"tf.Tensor(0.9251684, shape=(), dtype=float32)"
1006,31-08-2020 20:12,"tf.Tensor(0.968473, shape=(), dtype=float32)"
1007,31-08-2020 21:58,"tf.Tensor(0.9160188, shape=(), dtype=float32)"
1008,31-08-2020 23:31,"tf.Tensor(0.9893824, shape=(), dtype=float32)"


In [172]:
from sklearn import svm
results = pd.read_csv(r"C:\Users\Manjusha Pattadkal\Documents\ML Project\sentiment_percent_vector.csv")
clf = svm.SVC()
clf.fit(results.iloc[:,1:17], results.iloc[:,17])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [171]:
results.iloc[:,1:17]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.804878,0.195122,0.887218,0.112782,0.798246,0.201754,0.794326,0.205674,0.927536,0.072464,0.747826,0.252174,0.754839,0.245161,0.884848,0.115152
1,0.846154,0.153846,0.806452,0.161290,0.916667,0.083333,1.000000,0.000000,0.538462,0.461538,0.875000,0.125000,0.842105,0.157895,0.818182,0.181818
2,0.545455,0.454545,0.560000,0.440000,0.629371,0.363636,0.643750,0.356250,0.676768,0.323232,0.680312,0.319688,0.674324,0.325676,0.689531,0.310469
3,0.666667,0.000000,0.800000,0.200000,0.500000,0.500000,0.750000,0.250000,0.600000,0.400000,0.916667,0.083333,0.800000,0.200000,0.785714,0.214286
4,0.571429,0.428571,0.572368,0.421053,0.621429,0.378571,0.565217,0.434783,0.685864,0.314136,0.589147,0.410853,0.658182,0.341818,0.565517,0.434483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.808824,0.176471,0.750000,0.250000,0.850000,0.150000,0.470588,0.529412,0.685714,0.314286,0.846154,0.153846,0.725000,0.275000,0.722222,0.277778
197,0.785455,0.210909,0.631579,0.368421,0.780000,0.220000,0.691589,0.308411,0.757009,0.242991,0.748344,0.251656,0.760274,0.239726,0.760736,0.239264
198,0.728814,0.254237,0.723077,0.276923,0.764706,0.235294,0.731343,0.268657,0.761905,0.238095,0.660377,0.339623,0.687023,0.312977,0.692308,0.307692
199,0.602941,0.389706,0.582090,0.417910,0.627451,0.372549,0.625000,0.375000,0.589041,0.410959,0.500000,0.500000,0.518519,0.481481,0.645390,0.354610
